<a href="https://colab.research.google.com/github/allisonpg12/analitica3proyecto3/blob/main/CASO_SALUD_APG_AVRQ_MIVZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CASO DE ESTUDIO SALUD**

## **Importar librerias**

In [2]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, RidgeCV, Lasso
from functools import reduce
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.feature_selection import SelectFromModel

%matplotlib inline
from sklearn import tree
from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
import sqlite3
from datetime import datetime

# Evitar salidas de Warnings
import warnings
warnings.filterwarnings("ignore")

## **Lectura de datos - Procesamiento de bases**

In [3]:
#conectar el notebook al drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Lectura de bases de datos desde drive
df_cr = pd.read_excel('/content/drive/MyDrive/Trabajos grupo Analítica III/Trabajo 3 - Salud/RETO_df_cronicos.xlsx')
df_eg = pd.read_excel('/content/drive/MyDrive/Trabajos grupo Analítica III/Trabajo 3 - Salud/RETO_df_egresos.xlsx')
df_us = pd.read_excel('/content/drive/MyDrive/Trabajos grupo Analítica III/Trabajo 3 - Salud/RETO_df_usuarios.xlsx')

In [5]:
# Verificar tamaño de las bases de datos
print('Tamaño de la base de crónico:',df_cr.shape)
print('Tamaño de la base de egresos:',df_eg.shape)
print('Tamaño de la base de usuarios:',df_us.shape)

Tamaño de la base de crónico: (38717, 290)
Tamaño de la base de egresos: (6376, 1)
Tamaño de la base de usuarios: (183911, 1)


In [6]:
# Separación de columnas bases de datos de egresos y usuarios
# Usuarios
col =df_us.columns.tolist()
cols = col[0].split(',')
df_us[cols] =df_us[df_us.columns[0]].str.split(',', expand=True)
df_us.drop(columns=df_us.columns[0],inplace=True)
df_us.replace("", np.nan, inplace = True)
df_us.replace("Sin informaciÃ³n", np.nan, inplace = True)
df_us.replace("Sin Informacion", np.nan, inplace = True)
df_us.replace("Sin informacion en base de datos", np.nan, inplace = True)

# Egresos
df_eg.iloc[:,0] = df_eg.iloc[:,0].str.replace(', ', '')
col =df_eg.columns.tolist()
cols = col[0].split(',')
df_eg[cols] =df_eg[df_eg.columns[0]].str.split(',', expand=True)
df_eg.drop(columns=df_eg.columns[0],inplace=True)
df_eg.replace("", np.nan, inplace = True)
df_eg.replace("Sin informaciÃ³n", np.nan, inplace = True)
df_eg.replace("Sin Informacion", np.nan, inplace = True)
df_eg.replace("Sin informacion en base de datos", np.nan, inplace = True)
df_eg.replace("Sin informacion en Base de datos", np.nan, inplace = True)

In [7]:
# Verificar que las bases hayan cargado de forma correcta
df_us.head()

,MES,YEAR,NRODOC,SEXO,FECHA NACIMIENTO,EDAD,DEPARTAMENTO,MUNICIPIO,BARRIO,FECHA INICIO AL PGP,FECHA PRIMERA CLASE FUNCIONAL,PRIMERA CLASE FUNCIONAL,FECHA ÃšLTIMA CLASE FUNCIONAL,ÃšLTIMA CLASE FUNCIONAL,CICLO_VITAL,QUINQUENIO
0,ABRIL,2017,306221,F,1938-09-09 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABRIL,2017,306222,F,1974-07-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABRIL,2017,306223,F,1930-08-24 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ABRIL,2017,306224,F,1979-12-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABRIL,2017,306225,M,1943-06-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df_eg.head()

,YEAR,MES,NRO ATENCION,NRO INGRESO,SERVICIO HABILITADO COD,SERVICIO HABILITADO,TIPO IDENTIFICACION,NRODOC,FECHA NACIMIENTO,REGIMEN AFILIACION,...,DX PRINCIPAL EGRESO CAPITULO COD,DX PRINCIPAL EGRESO CAPITULO,CAUSA BASICA MUERTE COD,CAUSA BASICA MUERTE,CAUSA BASICA CATEGORIA COD,CAUSA BASICA CATEGORIA,CAUSA BASICA CAPITULO COD,CAUSA BASICA CAPITULO,PROFESIONAL ESPECIALIDAD,PROFESIONAL ESPECIALIDAD GRD
0,2017,MAYO,3433964,1899917,-1.0,NaN,CC,308185,1937-07-18 00:00:00.000,NaN,...,J00-J99,ENFERMEDADES DEL SISTEMA RESPIRATORIO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Medicina Interna
1,2017,DICIEMBRE,3720631,2018136,-1.0,NaN,CC,308185,1937-07-18 00:00:00.000,NaN,...,J00-J99,ENFERMEDADES DEL SISTEMA RESPIRATORIO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Medicina Interna
2,2018,FEBRERO,3912821,2159592,501.0,Urgencias >= 24 horas,CC,308185,1937-07-18 00:00:00.000,Contributivo,...,J00-J99,ENFERMEDADES DEL SISTEMA RESPIRATORIO,NaN,NaN,NaN,NaN,NaN,NaN,Medicina Interna,Urgentologia
3,2018,MAYO,4023613,2254075,501.0,Urgencias >= 24 horas,CC,308185,1937-07-18 00:00:00.000,Contributivo,...,I00-I99,ENFERMEDADES DEL SISTEMA CIRCULATORIO,NaN,NaN,NaN,NaN,NaN,NaN,Medicina Interna,Urgentologia
4,2018,JULIO,4080166,2292863,101.0,General adultos,CC,308185,1937-07-18 00:00:00.000,Contributivo,...,I00-I99,ENFERMEDADES DEL SISTEMA CIRCULATORIO,NaN,NaN,NaN,NaN,NaN,NaN,Medicina Interna,Medicina Interna


In [9]:
df_cr.head()

,YEAR,MES,Fecha diligenciamiento,Hora,AtenciÃ³n,Ingreso,Tipo,NRODOC,Peso,Talla,...,Diagnostico79,NombreDiagnostico79,Diagnostico80,NombreDiagnostico80,Diagnostico81,NombreDiagnostico81,Diagnostico82,NombreDiagnostico82,Diagnostico83,NombreDiagnostico83
0,2017,OCTUBRE,2017-10-02,9:00:20 a. m.,3912499,2629714,CC,311144,69.0,155.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017,OCTUBRE,2017-10-02,9:04:33 a. m.,3942247,2629726,CC,308418,62.0,155.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017,OCTUBRE,2017-10-02,9:11:43 a. m.,3898560,2629751,CC,308573,89.0,163.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017,OCTUBRE,2017-10-02,9:33:34 a. m.,3912599,2629828,CC,307549,88.0,163.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017,OCTUBRE,2017-10-02,9:43:59 a. m.,3906820,2629717,CC,307044,58.0,158.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#verifciar formato de variables y datos faltantes
print(df_us.info())
print(df_eg.info())
print(df_cr.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183911 entries, 0 to 183910
Data columns (total 16 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   MES                            183911 non-null  object
 1   YEAR                           183911 non-null  object
 2   NRODOC                         183911 non-null  object
 3   SEXO                           183901 non-null  object
 4   FECHA NACIMIENTO               111831 non-null  object
 5   EDAD                           149755 non-null  object
 6   DEPARTAMENTO                   82284 non-null   object
 7   MUNICIPIO                      81397 non-null   object
 8   BARRIO                         115344 non-null  object
 9   FECHA INICIO AL PGP            125658 non-null  object
 10  FECHA PRIMERA CLASE FUNCIONAL  87961 non-null   object
 11  PRIMERA CLASE FUNCIONAL        89641 non-null   object
 12  FECHA ÃšLTIMA CLASE FUNCIONAL  87985 non-nul

In [ ]:
# Eliminar columnas que tienen más del 50% de los datos nulos en la BD usuarios o que no aportan información
df_us.drop(['FECHA NACIMIENTO','DEPARTAMENTO', 'MUNICIPIO', 'BARRIO','FECHA INICIO AL PGP', 'FECHA PRIMERA CLASE FUNCIONAL', 'PRIMERA CLASE FUNCIONAL', 'FECHA ÃšLTIMA CLASE FUNCIONAL', 'QUINQUENIO', 'CICLO_VITAL'], axis=1, inplace = True)

In [13]:
# Eliminar datos faltantes de acuerdo a la clase funcional y la edad
df_us.dropna(subset=['ÃšLTIMA CLASE FUNCIONAL', 'EDAD'], inplace = True)

In [ ]:
# Cambiar nombre de columnas y tratar base final de usuarios
df_us.rename(columns={'ÃšLTIMA CLASE FUNCIONAL': 'clase_funcional'}, inplace=True)
df_us.columns = df_us.columns.str.lower()
df_us['clase_funcional'] = df_us['clase_funcional'].str.replace('CLASE FUNCIONAL ', '')
df_us['clase_funcional'] = df_us['clase_funcional'].str.replace('clase funcional ', '')
df_us['clase_funcional'] = df_us['clase_funcional'].str.replace('Clase funcional ', '')
df_us['edad'] = (df_us['edad'].astype(float)).astype(int)
df_us['year'] = df_us['year'].astype(int)
df_us['nrodoc'] = df_us['nrodoc'].astype(int)
df_us.head()

,mes,year,nrodoc,sexo,edad,clase_funcional
29233,OCTUBRE,2017,311028,M,65,3
29234,OCTUBRE,2017,310961,M,71,2B
29235,OCTUBRE,2017,310240,M,64,3
29237,OCTUBRE,2017,311235,F,81,2A
29239,OCTUBRE,2017,309710,F,86,4


In [ ]:
# Información Base de Datos Final de Usuarios
df_us.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125225 entries, 29233 to 183910
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   mes              125225 non-null  object
 1   year             125225 non-null  int64 
 2   nrodoc           125225 non-null  int64 
 3   sexo             125225 non-null  object
 4   edad             125225 non-null  int64 
 5   clase_funcional  125225 non-null  object
dtypes: int64(3), object(3)
memory usage: 6.7+ MB


In [ ]:
# Eliminar columnas que tienen más del 50% de los datos nulos en la BD egresos o que no aportan información
df_eg.drop(['NRO ATENCION', 'NRO INGRESO', 'SERVICIO HABILITADO COD', 'TIPO IDENTIFICACION', 'FECHA NACIMIENTO', 'FUENTE FINANCIACION1', 'FUENTE FINANCIACION2', 'BLOQUE ANTERIOR', 'PISO ANTERIOR',
            'SERVICIO ANTERIOR', 'NRO CAMA ANTERIOR', 'SERVICIO EGRESO', 'FECHA INGRESO SERVICIO', 'FECHA POSIBLE ALTA', 'FECHA ALTA MEDICA', 'FECHA RECAUDO', 'FECHA CAMILLERO', 'FECHA ENFERMERIA',
            'FECHA FACTURACION AUDIFARMA', 'FECHA FARMACIA', 'FECHA ASEO', 'TRANSFUSION SANGRE', 'ALTA MEDICA', 'POSIBLE ALTA', 'DXPRINCIPAL EGRESO', 'DX RELACIONADO1', 'DX RELACIONADO2', 'DX RELACIONADO3',
            'TIPO DIAGNOSTICO PRINCIPAL', 'PERTINENCIA DIAGNOSTICA', 'DX PRINCIPAL EGRESO CATEGORIA COD', 'DX PRINCIPAL EGRESO CATEGORIA', 'DX PRINCIPAL EGRESO CAPITULO', 'CAUSA BASICA MUERTE COD', 'CAUSA BASICA MUERTE',
            'CAUSA BASICA CATEGORIA COD', 'CAUSA BASICA CATEGORIA','CAUSA BASICA CAPITULO COD','CAUSA BASICA CAPITULO', 'PROFESIONAL ESPECIALIDAD GRD', 'DX RELACIONADO1 COD', 'DX RELACIONADO2 COD', 'DX RELACIONADO3 COD', 'DX PRINCIPAL EGRESO CAPITULO COD'], axis=1, inplace = True)

In [ ]:
# Eliminar datos faltantes de acuerdo a la fecha de ingreso, EPS validada, servicio admite
df_eg.dropna(subset=['FECHA INGRESO CLINICA', 'EPS VALIDADA', 'SERVICIO ADMITE'], inplace = True)

In [ ]:
# Cambiar nombre de columnas y tratar base final de egresos
df_eg.rename(columns={'DEMORA SALIDA CLINICA (DIAS)': 'DEMORA SALIDA CLINICA'}, inplace=True)
df_eg.columns = df_eg.columns.str.lower()
df_eg['servicio admite'] = df_eg['servicio admite'].str.replace('Urgencias Generales SÃ³tano','Urgencias Sotano')
df_eg['servicio admite'] = df_eg['servicio admite'].str.replace('CirugÃ­a', 'Cirugia')
df_eg['servicio admite'] = df_eg['servicio admite'].str.replace('HospitalizaciÃ³n','Hispitalizacion')
df_eg['servicio admite'] = df_eg['servicio admite'].str.replace('AtenciÃ³n Domiciliaria', 'Atencion Domiciliaria')
df_eg['servicio admite'] = df_eg['servicio admite'].str.replace('AtenciÃ³n Prioritaria', 'Atencion Prioritaria' )
df_eg['tipo egreso'] = df_eg['tipo egreso'].str.replace('Alta MÃ©dica', 'Alta Medica')
df_eg['year'] = df_eg['year'].astype(int)
df_eg['nrodoc'] = df_eg['nrodoc'].astype(int)
df_eg.head()

,year,mes,servicio habilitado,nrodoc,regimen afiliacion,eps validada,modalidad contrato,servicio admite,via ingreso,bloque,...,nro cama,tipo egreso,fecha ingreso clinica,fecha salida,demora asignacion cama,demora aplicacion medicamento,demora salida clinica,antibiotico,dxprincipal egreso cod,profesional especialidad
2,2018,FEBRERO,Urgencias >= 24 horas,308185,Contributivo,NUEVA EPS S.A. PGP,PGP,Urgencias Sotano,Urgencias,Bloque 3,...,FST101,Alta Medica,2018-02-21 05:21:03.340,2018-02-22 13:21:18.000,0:00,-27:20,0.0,SI,J441,Medicina Interna
3,2018,MAYO,Urgencias >= 24 horas,308185,Contributivo,NUEVA EPS S.A. PGP,PGP,Urgencias Sotano,Urgencias,Bloque 3,...,ERA109,Alta Medica,2018-05-19 15:07:11.027,2018-05-20 16:14:41.000,00:00,00:00,0.0,NO,I110,Medicina Interna
4,2018,JULIO,General adultos,308185,Contributivo,NUEVA EPS S.A. PGP,PGP,Urgencias Sotano,Urgencias,Bloque 1,...,711BS,Alta Medica,2018-07-26 12:44:55.287,2018-07-31 14:40:30.000,00:00,00:00,0.0,NO,I110,Medicina Interna
5,2018,OCTUBRE,General adultos,308185,Contributivo,NUEVA EPS S.A. PGP,PGP,Urgencias Sotano,Urgencias,Bloque 1,...,408AS,Alta Medica,2018-10-21 20:04:59.030,2018-10-27 14:58:20.000,00:00,00:00,0.0,NO,K573,Medicina Interna
6,2018,NOVIEMBRE,Urgencias >= 24 horas,308185,Contributivo,NUEVA EPS S.A. PGP,PGP,Urgencias Sotano,Urgencias,Bloque 3,...,ERA110,Alta Medica,2018-11-20 02:01:23.783,2018-11-22 09:22:53.000,00:00,00:00,0.0,NO,J441,Cirugia General


In [ ]:
# Información Base de Datos de Egresos
df_eg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4089 entries, 2 to 6375
Data columns (total 22 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   year                           4089 non-null   int64 
 1   mes                            4089 non-null   object
 2   servicio habilitado            4089 non-null   object
 3   nrodoc                         4089 non-null   int64 
 4   regimen afiliacion             4089 non-null   object
 5   eps validada                   4089 non-null   object
 6   modalidad contrato             4089 non-null   object
 7   servicio admite                4089 non-null   object
 8   via ingreso                    4089 non-null   object
 9   bloque                         4089 non-null   object
 10  unidad estrategica             4089 non-null   object
 11  piso                           4089 non-null   object
 12  nro cama                       4089 non-null   object
 13  tip

In [ ]:
# Cambiar tipos de variables de fechas y tratar variable demoras
df_eg['fecha salida']=pd.to_datetime(df_eg['fecha salida'])
df_eg['fecha ingreso clinica']=pd.to_datetime(df_eg['fecha ingreso clinica'])
df_eg['demora salida clinica'] = (df_eg['demora salida clinica'].astype(float))*24

# Demora asignación cama
demora = df_eg['demora asignacion cama'].str.split(':', expand=True)
demora.columns=['min','seg']
demora['min'] = demora['min'].astype(int)
demora['seg'] = demora['seg'].astype(int)
demora['min']=demora['min']/60
demora['seg']=demora['seg']/3600
df_eg['demora asignacion cama']=demora['min']+demora['seg']

# Demora asignación medicamento
demora = df_eg['demora aplicacion medicamento'].str.split(':', expand=True)
demora.columns=['min','seg']
demora['min'] = demora['min'].astype(int)
demora['seg'] = demora['seg'].astype(int)
demora['min']=demora['min']/60
demora['seg']=demora['seg']/3600
df_eg['demora aplicacion medicamento']=-1*(demora['min']+demora['seg'])

df_eg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4089 entries, 2 to 6375
Data columns (total 22 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   year                           4089 non-null   int64         
 1   mes                            4089 non-null   object        
 2   servicio habilitado            4089 non-null   object        
 3   nrodoc                         4089 non-null   int64         
 4   regimen afiliacion             4089 non-null   object        
 5   eps validada                   4089 non-null   object        
 6   modalidad contrato             4089 non-null   object        
 7   servicio admite                4089 non-null   object        
 8   via ingreso                    4089 non-null   object        
 9   bloque                         4089 non-null   object        
 10  unidad estrategica             4089 non-null   object        
 11  piso             

In [ ]:
# Seleccionar columnas de interés de la BD crónico
df_cr = pd.DataFrame(df_cr[['YEAR', 'MES', 'NRODOC','IMC', 'SaturaciÃ³n de OxÃ­geno (%)',
                            'PresiÃ³n Arterial SistÃ³lica', 'PresiÃ³n Arterial DiastÃ³lica', 'Frecuencia CardÃ­aca en reposo',
                            'METS -Ãndice MetabÃ³lico', 'VO2 - MÃ¡xima Cantidad de OxÃ­geno','Ãndice de Fragilidad Groningen',
                            'Glicemia', 'Hemoglobina Glicada', 'Lipoproteina', 'HDL', 'Colesterol total', 'Trigliceridos',
                            'Creatinina 1 consulta', 'Tasa de FiltraciÃ³n Glomerular TFG', 'Microalbuminuria',
                            'Hormona Estimulante de la Tiroides (TSH)', 'Creatinina 2 consulta']])

In [ ]:
# Cambiar nombre de columnas y tratar base final de crónicos
df_cr.replace("", np.nan, inplace = True)
df_cr.replace("Sin informaciÃ³n", np.nan, inplace = True)
df_cr.replace("Sin Informacion", np.nan, inplace = True)
df_cr.replace("Sin informacion en base de datos", np.nan, inplace = True)
df_cr.rename(columns={'SaturaciÃ³n de OxÃ­geno (%)': 'saturacion oxigeno'}, inplace=True)
df_cr.rename(columns={'PresiÃ³n Arterial SistÃ³lica': 'presion art sistolica'}, inplace=True)
df_cr.rename(columns={'PresiÃ³n Arterial DiastÃ³lica': 'presion art diastolica'}, inplace=True)
df_cr.rename(columns={'Frecuencia CardÃ­aca en reposo': 'frec cardiaca reposo'}, inplace=True)
df_cr.rename(columns={'METS -Ãndice MetabÃ³lico': 'indice metabolico'}, inplace=True)
df_cr.rename(columns={'VO2 - MÃ¡xima Cantidad de OxÃ­geno': 'max cant oxigeno'}, inplace=True)
df_cr.rename(columns={'Ãndice de Fragilidad Groningen': 'indice de fragilidad groingen'}, inplace=True)
df_cr.rename(columns={'Tasa de FiltraciÃ³n Glomerular TFG': 'tasa filtracion glomerular tfg'}, inplace=True)
df_cr.columns = df_cr.columns.str.lower()

In [ ]:
# Eliminar datos faltantes de acuerdo a la cantidad de datos que tienen los exámenes de laboratorio completos
df_cr.dropna(subset=['hdl','indice de fragilidad groingen', 'saturacion oxigeno'], inplace=True)

In [ ]:
# Información Base de Datos Final de Crónicos
df_cr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37489 entries, 0 to 38716
Data columns (total 22 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   year                                      37489 non-null  int64  
 1   mes                                       37489 non-null  object 
 2   nrodoc                                    37489 non-null  int64  
 3   imc                                       37489 non-null  float64
 4   saturacion oxigeno                        37489 non-null  float64
 5   presion art sistolica                     37489 non-null  int64  
 6   presion art diastolica                    37489 non-null  int64  
 7   frec cardiaca reposo                      37489 non-null  int64  
 8   indice metabolico                         37489 non-null  float64
 9   max cant oxigeno                          37489 non-null  float64
 10  indice de fragilidad groingen     

In [ ]:
# Calcular la diferencia entre la salida y el ingreso para obtener la estancia hospitalaria en horas
df_eg['est_hosp'] = (df_eg['fecha salida']-df_eg['fecha ingreso clinica']).dt.total_seconds() / 3600
df_eg.drop(['fecha ingreso clinica', 'fecha salida'], axis=1, inplace = True)

In [ ]:
# Visualizar estancia hospitalaria en horas
df_eg.head()

,year,mes,servicio habilitado,nrodoc,regimen afiliacion,eps validada,modalidad contrato,servicio admite,via ingreso,bloque,...,piso,nro cama,tipo egreso,demora asignacion cama,demora aplicacion medicamento,demora salida clinica,antibiotico,dxprincipal egreso cod,profesional especialidad,est_hosp
2,2018,FEBRERO,Urgencias >= 24 horas,308185,Contributivo,NUEVA EPS S.A. PGP,PGP,Urgencias Sotano,Urgencias,Bloque 3,...,Urgencias,FST101,Alta Medica,0.0,0.444444,0.0,SI,J441,Medicina Interna,32.004072
3,2018,MAYO,Urgencias >= 24 horas,308185,Contributivo,NUEVA EPS S.A. PGP,PGP,Urgencias Sotano,Urgencias,Bloque 3,...,Urgencias,ERA109,Alta Medica,0.0,-0.000000,0.0,NO,I110,Medicina Interna,25.124993
4,2018,JULIO,General adultos,308185,Contributivo,NUEVA EPS S.A. PGP,PGP,Urgencias Sotano,Urgencias,Bloque 1,...,Piso 7,711BS,Alta Medica,0.0,-0.000000,0.0,NO,I110,Medicina Interna,121.926309
5,2018,OCTUBRE,General adultos,308185,Contributivo,NUEVA EPS S.A. PGP,PGP,Urgencias Sotano,Urgencias,Bloque 1,...,Piso 4,408AS,Alta Medica,0.0,-0.000000,0.0,NO,K573,Medicina Interna,138.889158
6,2018,NOVIEMBRE,Urgencias >= 24 horas,308185,Contributivo,NUEVA EPS S.A. PGP,PGP,Urgencias Sotano,Urgencias,Bloque 3,...,Urgencias,ERA110,Alta Medica,0.0,-0.000000,0.0,NO,J441,Cirugia General,55.358116


Para la unión de las bases de datos, se toma como base de datos principal la que contiene los egresos de los pacientes. Asimismo, la variable clave para hacer la unión será 'nrodoc', para ello, es necesario verificar si cada dato contiene un código distinto o si hay códigos repetidos para varios datos.

In [ ]:
# Unión de bases de datos tomando como base principal egresos
print(len(df_eg['nrodoc'].unique()))
print(len(df_cr['nrodoc'].unique()))
print(len(df_us['nrodoc'].unique()))

print(len(df_eg['year'].unique()))
print(len(df_cr['year'].unique()))
print(len(df_us['year'].unique()))

print(len(df_eg['mes'].unique()))
print(len(df_cr['mes'].unique()))
print(len(df_us['mes'].unique()))

2418
6029
7019
3
4
4
12
12
12


Teniendo en cuenta que hay repetición de códigos nrodoc en la base de datos de egresos, se debe hacer la unión con esta variable, el año y el mes, con el fin de construir una base de datos final adecuada.

In [ ]:
# Unir las bases de datos por medio de key:
data_frames =[df_eg, df_us, df_cr]
df = reduce(lambda  left,right: pd.merge(left,right, on=['nrodoc', 'year','mes'],
                                            how='inner'), data_frames)

df.head()

,year,mes,servicio habilitado,nrodoc,regimen afiliacion,eps validada,modalidad contrato,servicio admite,via ingreso,bloque,...,hemoglobina glicada,lipoproteina,hdl,colesterol total,trigliceridos,creatinina 1 consulta,tasa filtracion glomerular tfg,microalbuminuria,hormona estimulante de la tiroides (tsh),creatinina 2 consulta
0,2018,JULIO,General adultos,308185,Contributivo,NUEVA EPS S.A. PGP,PGP,Urgencias Sotano,Urgencias,Bloque 1,...,5.40,100.0,53.0,176.0,111.0,1.00,40.40,9.90,2.81,0.00
1,2018,OCTUBRE,General adultos,308185,Contributivo,NUEVA EPS S.A. PGP,PGP,Urgencias Sotano,Urgencias,Bloque 1,...,5.40,100.0,53.0,176.0,111.0,1.00,39.01,9.90,2.81,0.00
2,2018,OCTUBRE,General adultos,308738,Contributivo,NUEVA EPS S.A. PGP,PGP,Urgencias Sotano,Urgencias,Bloque 1,...,6.50,55.2,33.2,116.0,138.0,1.12,38.26,0.00,1286.00,0.96
3,2018,DICIEMBRE,General adultos,308418,Contributivo,NUEVA EPS S.A. PGP,PGP,Hispitalizacion,Remitido,Bloque 1,...,6.08,86.0,44.0,153.0,110.0,1.00,54.58,8.30,4.65,1.00
4,2018,OCTUBRE,General adultos,311433,Contributivo,NUEVA EPS S.A. PGP,PGP,Hispitalizacion,Remitido,Bloque 1,...,5.80,100.0,35.5,188.0,262.0,1.00,81.60,3.17,0.00,0.00


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 777 entries, 0 to 776
Data columns (total 43 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   year                                      777 non-null    int64  
 1   mes                                       777 non-null    object 
 2   servicio habilitado                       777 non-null    object 
 3   nrodoc                                    777 non-null    int64  
 4   regimen afiliacion                        777 non-null    object 
 5   eps validada                              777 non-null    object 
 6   modalidad contrato                        777 non-null    object 
 7   servicio admite                           777 non-null    object 
 8   via ingreso                               777 non-null    object 
 9   bloque                                    777 non-null    object 
 10  unidad estrategica                    

In [ ]:
# Seleccionar la modalidad de contrato PGP
df = df.loc[df['modalidad contrato']=='PGP']
df.drop(['modalidad contrato'], axis=1, inplace = True)
df.year = df['year'].astype(str)
df.nrodoc = df['nrodoc'].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 758 entries, 0 to 776
Data columns (total 42 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   year                                      758 non-null    object 
 1   mes                                       758 non-null    object 
 2   servicio habilitado                       758 non-null    object 
 3   nrodoc                                    758 non-null    object 
 4   regimen afiliacion                        758 non-null    object 
 5   eps validada                              758 non-null    object 
 6   servicio admite                           758 non-null    object 
 7   via ingreso                               758 non-null    object 
 8   bloque                                    758 non-null    object 
 9   unidad estrategica                        758 non-null    object 
 10  piso                                  